In [1]:
from utils import RenderJSON

In [2]:
import streamlit as st
from congress import Congress
# from deta import Deta

PROPUBLICA_API_KEY = st.secrets["PROPUBLICA_API_KEY"];

# Chamber
SENATE = 'senate'
HOUSE = 'house'
BOTH = 'both' # idk if this works for everything, only votes

congress = Congress(PROPUBLICA_API_KEY)

In [3]:
@st.cache_data
def download_all_passed_bills(congress_num, verbose=False):
    bills = []

    page=1
    passed_bills = congress.bills.passed(chamber=BOTH, congress=congress_num, page=1)
    while passed_bills['num_results'] > 0:
        if verbose:
            print(f"{len(bills)} downloaded")

        bills.extend(passed_bills['bills'])
        page += 1
        passed_bills = congress.bills.passed(chamber=BOTH, congress=congress_num, page=page)

    if verbose:
        print(f"{len(bills)} bills downloaded from {bills[-1]['last_vote']} to {bills[0]['last_vote']}")
    return bills

bills_117 = download_all_passed_bills(117, verbose=False)
len(bills_117)

2023-03-06 05:41:08.353 
  command:

    streamlit run /home/nzarding/miniconda3/envs/billbriefs/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


306

In [16]:
# @st.cache_data
# def get_bill_chamber_vote_data(chamber, date, bill_id, verbose=False):
#     '''
#     Given a chamber, date, and bill_id: find the voting records of given date associated with given bill_id.
#     If multiple votes return are allowed.



#     house {'Passed'}
# senate {'Amendment Rejected', 'Motion Rejected', 'Amendment Agreed to', 'Resolution Agreed to', 'Cloture Motion Agreed to', 'Joint Resolution Passed', 'Motion Agreed to', 'Bill Passed'}
#     '''
#     votes = congress.votes.by_date(chamber, date)
#     # # After testing paging doesn't seem required.
#     # if votes['num_results'] >= 20: 
#     #     print(votes['num_results'], 'results, might need paging')
#     if verbose:
#         print("num of votes:", len(votes))

#     votes_with_any_billid = [ vote for vote in votes['votes'] if 'bill_id' in vote['bill'].keys() ]
#     votes_with_matching_billid = [vote for vote in votes_with_any_billid if vote['bill']['bill_id'] == bill_id]
#     if (len(votes_with_matching_billid) == 0):
#         return None


#     return [congress.votes.get( # Must I call this or is this included in the bill_vote_by_date?
#         chamber      = bill_vote_by_date['chamber'],
#         rollcall_num = bill_vote_by_date['roll_call'],
#         session      = bill_vote_by_date['session'],
#         congress     = bill_vote_by_date['congress'],
#     ) for bill_vote_by_date in votes_with_matching_billid]


# def get_bill_vote_data(passed_bill, verbose=False):
#     '''
#     Given a bill that passed, get the detailed voting data for both House and Senate.
#     '''
#     house_date  = passed_bill['house_passage']
#     senate_date = passed_bill['senate_passage']
#     bill_id     = passed_bill['bill_id']

#     if verbose:
#         print(f'Bill ID: {bill_id}, Passed House: {house_date}, Passed Senate: {senate_date}')

#         print('Getting all house votes on', house_date)
#     house_votes = get_bill_chamber_vote_data(HOUSE, house_date, bill_id, verbose)
#     # house_vote_positions = None if house_votes is None else house_votes['votes']['vote']['positions']

#     if verbose:
#         print('Getting all senate votes on', house_date)
#     senate_votes = get_bill_chamber_vote_data(SENATE, senate_date, bill_id, verbose)
#     # senate_vote_positions = None if senate_votes is None else senate_votes['votes']['vote']['positions']

#     return {
#         "bill_id": bill_id,
#         "house_passage": house_date,
#         "senate_passage": senate_date,
#         "house_votes": house_votes,
#         "senate_votes": senate_votes,
#         # "house_vote_positions": house_vote_positions,
#         # "senate_vote_positions": senate_vote_positions,
#         "latest_major_action": passed_bill["latest_major_action"]
#     }

In [4]:
# @st.cache_data
def get_bill_chamber_vote_data(chamber, date, bill_id, verbose=False):
    '''
    Given a chamber, date, and bill_id: find the voting records of given date associated with given bill_id.
    If multiple votes return are allowed.

    vote result values
    house: {'Passed'}
    senate: {'Amendment Rejected', 'Motion Rejected', 'Amendment Agreed to', 'Resolution Agreed to', 'Cloture Motion Agreed to', 'Joint Resolution Passed', 'Motion Agreed to', 'Bill Passed'}
    '''
    PASSING = ['Amendment Agreed to', 'Resolution Agreed to', 'Bill Passed', # Senate
               'Passed']                                                     # House

    votes = congress.votes.by_date(chamber, date)
    # # After testing paging doesn't seem required.
    # if votes['num_results'] >= 20: 
    #     print(votes['num_results'], 'results, might need paging')
    if verbose:
        print("num of votes:", len(votes))

    votes_with_any_billid = [ vote for vote in votes['votes'] if 'bill_id' in vote['bill'].keys() ]
    votes_with_matching_billid = [vote for vote in votes_with_any_billid if vote['bill']['bill_id'] == bill_id]
    votes_with_matching_billid_that_passed = [vote for vote in votes_with_matching_billid if vote['result'] in PASSING]

    if (len(votes_with_matching_billid_that_passed) == 0):
        return None


    return [congress.votes.get( # Must I call this or is this included in the bill_vote_by_date?
        chamber      = bill_vote_by_date['chamber'],
        rollcall_num = bill_vote_by_date['roll_call'],
        session      = bill_vote_by_date['session'],
        congress     = bill_vote_by_date['congress'],
    ) for bill_vote_by_date in votes_with_matching_billid_that_passed]


def get_bill_vote_data(passed_bill, verbose=False):
    '''
    Given a bill that passed, get the detailed voting data for both House and Senate.
    '''
    house_date  = passed_bill['house_passage']
    senate_date = passed_bill['senate_passage']
    bill_id     = passed_bill['bill_id']

    if verbose:
        print(f'Bill ID: {bill_id}, Passed House: {house_date}, Passed Senate: {senate_date}')

        print('Getting all house votes on', house_date)
    house_votes = get_bill_chamber_vote_data(HOUSE, house_date, bill_id, verbose)
    # house_vote_positions = None if house_votes is None else house_votes['votes']['vote']['positions']

    if verbose:
        print('Getting all senate votes on', house_date)
    senate_votes = get_bill_chamber_vote_data(SENATE, senate_date, bill_id, verbose)
    # senate_vote_positions = None if senate_votes is None else senate_votes['votes']['vote']['positions']

    return {
        "bill_id": bill_id,
        "house_passage": house_date,
        "senate_passage": senate_date,
        "house_votes": house_votes,
        "senate_votes": senate_votes,
        # "house_vote_positions": house_vote_positions,
        # "senate_vote_positions": senate_vote_positions,
        "latest_major_action": passed_bill["latest_major_action"]
    }

In [5]:
RenderJSON(bills_117[0])

In [6]:
records = []
for i, bill in enumerate(bills_117[:]):
    # print('idx', i)
    res = get_bill_vote_data(bill)
    # if res is not None:
    records.append(res)

In [ ]:
RenderJSON(records)

In [9]:
senate_records = [record for record in records if record["senate_votes"] is not None]
RenderJSON(senate_records)

In [10]:
# is every bill_id unique?
len(set([ r['bill_id'] for r in records])) == len(records)

True

## Time to populate Deta DB
- https://deta.space/manual/features/collections
- https://deta.space/docs/en/basics/data#deta-collections
- https://deta.space/docs/en/reference/base/sdk

I still need to grab the whole result from passed bills. My DB will be an easy to access all-in-one.


DRAFT

db:
    - Passed Bills
        - congress_N
            - some_bill_id (unique from propub)
                - propub result
                - house_vote_data
                - senate_vote_data

In [11]:
from deta import Deta

DETA_API_KEY = st.secrets["DETA_API_KEY"];

deta = Deta(DETA_API_KEY)

In [12]:
# https://deta.space/docs/en/reference/base/sdk

help(deta.Base)

Help on method Base in module deta:

Base(name: str, host: str = None) method of deta.Deta instance



In [14]:
passed_bills_db = deta.Base("passed_bills")

In [15]:
for i, r in enumerate(records):
    print(f'record {i} out of {len(records) - 1}')
    passed_bills_db.put(
        data=r,
        key=r['bill_id']
    )

record 0 out of 306
record 1 out of 306
record 2 out of 306
record 3 out of 306
record 4 out of 306
record 5 out of 306
record 6 out of 306
record 7 out of 306
record 8 out of 306
record 9 out of 306
record 10 out of 306
record 11 out of 306
record 12 out of 306
record 13 out of 306
record 14 out of 306
record 15 out of 306
record 16 out of 306
record 17 out of 306
record 18 out of 306
record 19 out of 306
record 20 out of 306
record 21 out of 306
record 22 out of 306
record 23 out of 306
record 24 out of 306
record 25 out of 306
record 26 out of 306
record 27 out of 306
record 28 out of 306
record 29 out of 306
record 30 out of 306
record 31 out of 306
record 32 out of 306
record 33 out of 306
record 34 out of 306
record 35 out of 306
record 36 out of 306
record 37 out of 306
record 38 out of 306
record 39 out of 306
record 40 out of 306
record 41 out of 306
record 42 out of 306
record 43 out of 306
record 44 out of 306
record 45 out of 306
record 46 out of 306
record 47 out of 306
re

In [34]:
records[160]

{'bill_id': 'hr5376-117',
 'house_passage': '2022-08-12',
 'senate_passage': '2022-08-07',
 'house_votes': [{'votes': {'vote': {'congress': 117,
     'session': 2,
     'chamber': 'House',
     'roll_call': 420,
     'source': 'http://clerk.house.gov/evs/2022/roll420.xml',
     'url': 'http://clerk.house.gov/evs/2022/roll420.xml',
     'bill': {'bill_id': 'hr5376-117',
      'number': 'H.R.5376',
      'api_uri': 'https://api.propublica.org/congress/v1/117/bills/hr5376.json',
      'title': 'To provide for reconciliation pursuant to title II of S. Con. Res. 14.',
      'short_title': 'Inflation Reduction Act of 2022',
      'latest_action': 'Became Public Law No: 117-169.'},
     'amendment': {},
     'question': 'On Motion to Concur in the Senate Amendment',
     'question_text': '',
     'description': 'To provide for reconciliation pursuant to title II of S. Con. Res. 14',
     'vote_type': 'YEA-AND-NAY',
     'date': '2022-08-12',
     'time': '17:38:00',
     'result': 'Passed',
 

In [36]:
RenderJSON(records[159])